In [36]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import re
import json
import unicodedata
import numpy as np
import ast
from sklearn.impute import KNNImputer
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../src'))

from data_preparation_utils import load_data, drop_and_save_columns, clean_text, filter_rows_by_column_value
pd.set_option('display.max_columns', None)

# load/save data

In [5]:
# Load processed data
df = load_data(r"G:\My Drive\french_resturants\data\processed\french_resturants_only.csv")

In [ ]:
#save data frame
df.to_csv(r"G:\My Drive\french_resturants\data\processed\french_resturants_only.csv", index=False)

# finding columns with nulls

In [8]:
# Step 1: Find columns with at least one null
null_columns = df.columns[df.isnull().any()].tolist()

# Step 2: Create a subset DataFrame with only those columns
df_nulls = df[null_columns]

# Step 3: (Optional) Filter rows where at least one of those columns is actually null
df_nulls_only = df_nulls[df_nulls.isnull().any(axis=1)]

# Display summary
print(f"Found {len(null_columns)} columns with nulls.")
print(f"Subset has {df_nulls_only.shape[0]} rows with at least one null value.")

Found 47 columns with nulls.
Subset has 155223 rows with at least one null value.


In [10]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()
viz = AV.AutoViz(
    '',  # empty string means use a DataFrame directly
    dfte=df_nulls,
    depVar='',  # no target variable at this stage
)

Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)
    Since nrows is smaller than dataset, loading random sample of 150000 rows into pandas...
Shape of your Data Set loaded: (150000, 47)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  5
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  33
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  3
    Number of Numeric-Boolean Columns =  

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
region,object,0.001355,0,,,"2 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
province,object,11.351516,0,,,"16752 missing values. Impute them with mean, median, mode, or a constant value such as 123., 58 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
city,object,0.082670,9,,,"122 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
claimed,object,0.113163,0,,,"167 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
awards,object,71.663900,0,,,"105758 missing values. Impute them with mean, median, mode, or a constant value such as 123., 53 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: float, object,"
top_tags,object,7.556158,5,,,"11151 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
price_level,object,17.868880,0,,,"26370 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
price_range,object,65.751652,2,,,"97033 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
meals,object,33.059122,0,,,"48787 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,, Possible high cardinality column with 399 unique values: Use hash encoding or text embedding to reduce dimension."
cuisines,object,13.267152,8,,,"19579 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"


Number of All Scatter Plots = 15


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nl

Could not draw wordcloud plot for city. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.

Could not draw wordcloud plot for top_tags. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.

Could not draw wordcloud plot for cuisines. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://n

# nulls groups:

## ✅ Group A — Low Null % (under ~10%)

In [ ]:
# region

# city

# claimed

# moved_location

## 🟡 Group B — Moderate Null % (~10% to ~35%)

In [ ]:
# province

# top_tags

# cuisines

# meals

# default_language

## 🔴 Group C — High Null % (Over 50%)

In [ ]:
# awards

# special_diets

# features

# price_range

# mon_open_1st → sun_close_2nd (all time slot columns)

## 📈 Group D — Numerical Columns

In [ ]:
# open_days_per_week

# open_hours_per_week

# working_shifts_per_week

# rank

# total_restaurants_in_city

# null handling in group A

In [21]:
group_a_cols = ['region', 'city', 'claimed', 'moved_location']

for col in group_a_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# null handling in group B

In [26]:
# 📌 Group B: Moderate-null categorical columns
# Strategy:
# - Use "Unknown" for single-value categorical fields
# - Use "Missing" for multi-label/tag-like columns that will be encoded later

# Define how to fill each column
group_b_fill_map = {
    'province': 'Unknown',          # Discrete category
    'default_language': 'Unknown',  # Discrete category
    'top_tags': 'Missing',          # Multi-label (comma-separated tags)
    'cuisines': 'Missing',          # Multi-label (e.g., "French, Italian")
    'meals': 'Missing'              # Multi-label (e.g., "Lunch, Dinner")
}

# Apply the fill per column
for col, fill_value in group_b_fill_map.items():
    if col in df.columns:
        df[col] = df[col].fillna(fill_value)

# null handling in group C

## handling shifts columns

In [30]:
# Get all time slot columns
time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]

# Step 1: Identify rows where ALL time columns are null
all_time_slots_null = df[time_slot_cols].isnull().all(axis=1)

# Step 2: Fill partial nulls with "Closed" for others
for day in ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']:
    open_1st = f'{day}_open_1st'
    close_1st = f'{day}_close_1st'
    open_2nd = f'{day}_open_2nd'
    close_2nd = f'{day}_close_2nd'
    
    # Only proceed if the columns exist
    if all(col in df.columns for col in [open_1st, close_1st, open_2nd, close_2nd]):

        # Condition: row is NOT fully empty in time columns
        not_all_null = ~all_time_slots_null

        # Handle 1st shift nulls
        df.loc[not_all_null & df[open_1st].isnull(), open_1st] = 'Closed'
        df.loc[not_all_null & df[close_1st].isnull(), close_1st] = 'Closed'

        # Handle 2nd shift nulls — only fill if it’s not "continuous"
        is_continuous = df[open_2nd] == 'continuous'
        df.loc[not_all_null & df[open_2nd].isnull() & ~is_continuous, open_2nd] = 'Closed'
        df.loc[not_all_null & df[close_2nd].isnull() & ~is_continuous, close_2nd] = 'Closed'

In [34]:
df[time_slot_cols].head(30)

,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,09:00,14:30,continuous,continuous,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,22:00,09:00,14:30,19:00,22:00,09:00,16:00,continuous,continuous
6,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00
7,Closed,Closed,Closed,Closed,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous
8,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:00,continuous,continuous,12:00,14:00,continuous,continuous,12:00,14:00,19:00,21:00,19:00,21:00,continuous,continuous,12:00,14:00,19:00,21:00
9,Closed,Closed,Closed,Closed,09:00,16:00,continuous,continuous,09:00,16:00,continuous,continuous,09:00,21:00,continuous,continuous,09:00,21:00,continuous,continuous,16:45,23:45,continuous,continuous,09:00,17:00,continuous,continuous


In [38]:
# 1️⃣ Define your time slot columns
time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]

# 2️⃣ Store category encoders for each time column
category_maps = {}
for col in time_slot_cols:
    df[col] = df[col].astype('category')
    category_maps[col] = dict(enumerate(df[col].cat.categories))
    df[col] = df[col].cat.codes.replace(-1, np.nan)

# 3️⃣ Prepare container for results
df_imputed_parts = []

# 4️⃣ Group by (region, city) and apply KNN imputer
for group_key, group_df in df.groupby(['region', 'city']):
    # If group is all NaNs, skip and leave them as-is
    if group_df[time_slot_cols].isnull().all(axis=None):
        df_imputed_parts.append(group_df[time_slot_cols])
        continue

    imputer = KNNImputer(n_neighbors=3)
    imputed_array = imputer.fit_transform(group_df[time_slot_cols])
    imputed_df = pd.DataFrame(imputed_array, columns=time_slot_cols, index=group_df.index)
    df_imputed_parts.append(imputed_df)

# 5️⃣ Recombine all imputed groups
df_imputed_times = pd.concat(df_imputed_parts).sort_index()

# 6️⃣ Convert imputed numeric codes back to original categories
for col in time_slot_cols:
    code_to_label = category_maps[col]
    df[col] = df_imputed_times[col].round().astype('Int64').map(code_to_label)

print("✅ Imputation complete: original time slot columns have been replaced with KNN-imputed values.")

✅ Imputation complete: original time slot columns have been replaced with KNN-imputed values.
